# **Basic Data Science Projects using Python, NumPy, Pandas, Matplotlib, Regular Expressions, and SQL**

<center>

*By: Prof. James Abello, Haoyang Zhang*

*Computer Science Department*

*Rutgers University*

*Nov. 21, 2024.*

</center>

## Topic 2: Linear Regression (np, matplotlib)

#### **Objective:** Identify how well a linear regression line fits labor force participation rates between males and females. [World Bank data](https://www.worldbank.org/ext/en/home)

#### **Estimated Completion Time: 6 hours**

Linear regression is a widely used method for modeling the relationship between an output variable and one or more input variables.

In this project, we will focus on identifying the factors that influence the difference in labor force participation rates between males and females using the [World Bank data](https://www.worldbank.org/ext/en/home).

> A collection of good references of Introduction to Linear Regression is available at:
>
> - Statistics 101: Linear Regression, The Very Basics by Brandon Foltz: https://www.youtube.com/watch?v=ZkjP5RJLQF4
> - Statistics 101: Statistics 101: Linear Regression, Algebra, Equations, and Patterns: https://www.youtube.com/watch?v=iAgYLRy7e20
> - Statistics 101: Linear Regression, The Least Squares Method: https://www.youtube.com/watch?v=Qa2APhWjQPc
> - Statistics 101: Linear Regression, Fit and Coefficient of Determination: https://www.youtube.com/watch?v=kHZBy1uVNnM

### Tasks

#### Level 1

In this level, we will examine whether and to what extent a country's GDP per capita is related to the difference between male and female labor force participation rates.

##### Task 1.1

Write a Python function `get_data_1()` that retrieves the following data for each country/territory from the World Bank API:
- GDP per capita (current US$) (NY.GDP.PCAP.CD)
- Labor force participation rate, male (% of male population ages 15+) (modeled ILO estimate) (SL.TLF.CACT.MA.ZS)
- Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate) (SL.TLF.CACT.FE.ZS)

The link to the World Bank API is:
```python
f"https://api.worldbank.org/v2/country/{country_code}/indicator/NY.GDP.PCAP.CD?format=json&per_page=5000"
f"https://api.worldbank.org/v2/country/{country_code}/indicator/SL.TLF.CACT.MA.ZS?format=json&per_page=5000"
f"https://api.worldbank.org/v2/country/{country_code}/indicator/SL.TLF.CACT.FE.ZS?format=json&per_page=5000"
```
where `country_code` is a three-letter code for each country/territory, and you can find all of them at https://api.worldbank.org/v2/country?format=json&per_page=5000 with the key `id`.

```python
def get_data_1():
    """
    Retrieve the GDP per capita, labor force participation rate of each country/territory
    IN: None
    OUT: DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    """
    pass
```

In [ ]:
#libraries used
import requests
import json 
import pandas as pd
import time
from mysql.connector import connect, Error
#%matplotlib ipympl (for grader if you'd like to use interactive graphs, however it causes errors in the notebook issues on github)
from matplotlib import pyplot as plt
import numpy as np
import tkinter as tk


In [ ]:
def req_helper(url):
    '''
    IN:
        url: str, url to be fetched
    OUT:
        the list object returned by the fetch call
    '''

    #assertions
    assert type(url) is str

    res = requests.get(url)
    obj = json.loads(res.text)
    return obj[1]

def get_all_countries():
    '''
    IN:
        None
    OUT:
        list[tup()] : list of tuples of country (name,id) 
    '''
    country_name_id = []
    all_countries  = req_helper("https://api.worldbank.org/v2/country?format=json&per_page=5000")
    for obj in all_countries:
        #make sure the obj has the keys
        if obj.get("name") and obj.get("id"):
            country_name_id.append((obj["name"],obj["id"]))
    return country_name_id



def get_data_1():
    """
    Retrieve the GDP per capita, labor force participation rate of each country/territory
    IN: None
    OUT: DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    """
    #get all name and ids for country
    country_name_id = get_all_countries()

    data = {}
    indicators = {
                    "NY.GDP.PCAP.CD":"gdp",
                    "SL.TLF.CACT.MA.ZS":"ML",
                    "SL.TLF.CACT.FE.ZS":"FL"
                 }
    for j, (name,id)  in enumerate(country_name_id): #use the id to get gdp per capital 
        data[id] = [{
            "name":name,
             "id":id,
             "date":None,
             "gdp":None,
             "ML":None,
             "FL":None     
        }]
        #append to dictionary for each year for each country
        for indicator,col in indicators.items():
            country_data = req_helper(f"https://api.worldbank.org/v2/country/{id}/indicator/{indicator}?format=json&per_page=5000")
            if country_data != None:
                if len(data[id]) != len(country_data):
                        data[id].extend([{"name":name,"id":id,"date":None,"gdp":None,"ML":None,"FL":None} for _ in range(len(country_data) - len(data[id]))])
                for i,obj in enumerate(country_data):
                    #make sure values exist
                    if obj.get("date") and data[id][i]["date"] == None:
                        data[id][i]["date"] = obj["date"]
                    if obj.get("value"):
                        data[id][i][col] = obj["value"]
        #if j%10 == 0 : (for progress)
        #    print(j)

    #absolutely no data was found not surre if we should extend or delete the data entirely...(extending 4 now)
    cp = data.copy()
    length =  0
    for id,list_of_dicts in list(cp.items()):
        if len(list_of_dicts) > 1 and length ==0:
            length = len(list_of_dicts)
        if len(list_of_dicts) == 1:
            data[id] *= length
            
    list_of_data = [d for l in list(data.values()) for d in l]
 
    return pd.DataFrame(list_of_data)

In [ ]:
#create global data variable takes ~3min
df = get_data_1()

##### Task 1.2

Write a Python function `save_data_1()` that saves the data in a MySQL database. 

Your database should have the following tables:
- `country` with the columns `name`, `code`
- `gdp_per_capita` with the columns `country_code`, `year`, `value`
- `labor_force_participation_rate` with the columns `country_code`, `year`, `male_value`, `female_value`

```python
def save_data_1(data, mydb, cursor, db_name):
    """
    Save the data in a MySQL database
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: None
    """
    pass
```

In [ ]:
WAIT_TIME = 10
def connect_to_db(db_name):
    """
    Connect to the database
    IN: db_name, str, the name of the database in MySQL
    OUT: mydb, the database connection
    EXCEPTION: TimeoutError, if the database is not running
    """

    #assertions
    assert type(db_name) is str
    
    #try connecting 10 times then raise error if unable to connect
    num_of_tries = 0
    while num_of_tries < WAIT_TIME:
        try:
            mydb = connect(host="127.0.0.1", user="root", db=db_name)
            if mydb.is_connected():
                print("Connected")
                break
        except Error as e:
            print(e)

        num_of_tries += 1
        print("Waiting for MySQL to start")
        time.sleep(1)

    if num_of_tries >= WAIT_TIME:
        raise TimeoutError("MySQL is not running")

    return mydb


In [ ]:
db_name = "linreg"
mydb =  connect_to_db(db_name)
cursor = mydb.cursor()

In [ ]:
def create_tables_helper():
    '''
    IN:
        None
    OUT:
        tup(stmnt1,stmnt2,stmnt3)
    '''
    #hopefully varchar 50 is enough?
    #might not need a primary key
    create_country = '''
    create table country (
    name varchar(100) not null,
    code char(3) primary key 
    );
    '''

    #not sure if year should be year type or int,year can be null, AFR has no values
    create_gdp_per_capita = '''
    create table gdp_per_capita (
    country_code char(3) not null,
    year int,
    value double,
    foreign key (country_code) references country(code)
    );
    '''
    #year can be null, AFR has no values
    create_labor_force_participation_rate = '''
    create table labor_force_participation_rate(
    country_code char(3) not null,
    year int , 
    male_value double,
    female_value double,
    foreign key (country_code) references country(code)
    );
    '''
    return [create_country,create_gdp_per_capita,create_labor_force_participation_rate]

#note our database name is provided in the cell above
def save_data_1(data,mydb,cursor,db_name):
    """
    Save the data in a MySQL database
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: None

    `country` with the columns `name`, `code`
    `gdp_per_capita` with the columns `country_code`, `year`, `value`
    `labor_force_participation_rate` with the columns `country_code`, `year`, `male_value`, `female_value`
    
    """

    #assertions
    assert type(data) is pd.DataFrame
    assert type(db_name) is str 
   

    #********HANDLES CREATION OF TABLES********#
    create_statements =  create_tables_helper()
    #give it a few seconds so the country table can  be made so the other tables can reference it, 5 secs might b overkill  but doenst matter.
    for q in create_statements:
        cursor.execute(q) 
        time.sleep(2)
    #********HANDLES CREATION OF TABLES********#

    #********HANDLES INSERTION INTO COUNTRY********#
    unique_name_id = data[["name","id"]].drop_duplicates().values
    for name,id in unique_name_id:
        q= f'''
        insert into country (name,code) values ("{name}","{id}");
        '''
        cursor.execute(q) 
    #********HANDLES INSERTION INTO COUNTRY********#


    #********HANDLES INSERTION INTO GDP_PER_CAPITA********#
    filtered_df = data[["id","date","gdp"]]
    for _,row in filtered_df.iterrows():
        q= f'''
            insert into gdp_per_capita (country_code, year, value) values ('{row["id"]}',{'null' if pd.isnull(row["date"]) else row["date"]},{'null' if pd.isnull(row["gdp"]) else row["gdp"]});
            '''
        cursor.execute(q)
    #********HANDLES INSERTION INTO GDP_PER_CAPITA********#

    #********HANDLES INSERTION INTO LABOR_FORCE_PARTICIPATION_RATE********#
    filtered_df = data[["id","date","ML","FL"]]
    for _,row in filtered_df.iterrows():
            q = f"""
            insert into labor_force_participation_rate (country_code, year, male_value,female_value) values ('{row["id"]}',{'null' if pd.isnull(row["date"]) else row["date"]},{'null' if pd.isnull(row["ML"]) else row["ML"]}, {'null' if pd.isnull(row["FL"]) else row["FL"]});
            """
            cursor.execute(q)
    #********HANDLES INSERTION INTO LABOR_FORCE_PARTICIPATION_RATE********#

    mydb.commit()



In [ ]:
#check if it worked
save_data_1(df,mydb,cursor,db_name="linreg")


##### Task 1.3

Write a Python function `load_data_1()` that queries the data from the MySQL database.

Your query should return a table with the columns `country_name`, `country_code`, `year`, `gdp_per_capita`, `difference_labor_force_participation_rate`.

Note:

- Records with missing data should be ignored.
- The `difference_labor_force_participation_rate` should be calculated as `male_value` - `female_value`.

```python
def load_data_1(mydb, cursor, db_name):
    """
    Query the data from the MySQL database
    IN: mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    """
    pass
```

In [ ]:
def load_data_1(mydb, cursor, db_name):
    """
    Query the data from the MySQL database
    IN: mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    `country` with the columns `name`, `code`
    `gdp_per_capita` with the columns `country_code`, `year`, `value`
    `labor_force_participation_rate` with the columns `country_code`, `year`, `male_value`, `female_value`
    """
    #assertions
    assert type(db_name) is str
    
    #join tables, and essentially grab rows in which no col value is null
    q = f"""
    select C.name, C.code, G.year, G.value, (L.male_value - L.female_value) as 'difference_labor_force_participation_rate' 
    from country C 
    inner join gdp_per_capita G on C.code = G.country_code
    inner join labor_force_participation_rate L on C.code = L.country_code
    where G.year is not null and 
    G.value is not null and 
    L.male_value is not null and 
    L.female_value is not null and
    G.year = L.year
    """
    cursor.execute(q)
    res = cursor.fetchall()
    data = []
    for name,id,year,gdp,diff_m_f in res:
        d ={
            "country_name":name,
            "country_code":id,
            "year":year,
            "gdp_per_capita":gdp,
            "difference_labor_force_participation_rate":diff_m_f,
        }
        data.append(d)

    cleaned_df = pd.DataFrame(data)
    return cleaned_df


In [ ]:
cleaned_df = load_data_1(mydb,cursor,db_name)

##### Task 1.4

Write a Python function `plot_data_1()` that visualizes the relationship between GDP per capita and the difference in labor force participation rates.

- The x-axis should be the GDP per capita, and 
- The y-axis should be the difference of the labor force participation rate.

```python
def plot_data_1(data):
    """
    Visualize the relationship between GDP per capita and the difference in labor force participation rates
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    OUT: None
    """
    pass
```

In [ ]:
def plot_data_1(data):
    """
    Visualize the relationship between GDP per capita and the difference in labor force participation rates
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    OUT: None
    """

    #assertions
    assert type(data) is pd.DataFrame

    #grab appropriate cols
    x = data["gdp_per_capita"]
    y = data["difference_labor_force_participation_rate"]
    #for better viewing
    scale = [4] * len(y)
    plt.figure(figsize=(14,8))
    plt.scatter(x=x,y=y,s=scale)

    plt.xlabel("gdp per capita")
    plt.ylabel("diff in male and female labor")
    plt.show()


In [ ]:
plot_data_1(cleaned_df)

##### Task 1.5

Write a Python function `compute_linear_regression_1()` that fits a linear regression line to the data, with GDP per capita as the predictor and the difference in labor force participation rate as the response. Your function should return the slope and the intercept of the regression line.

The linear regression line can be found by minimizing the sum of the squared differences between the observed and predicted values. The slope and intercept can be calculated as follows:
$$
\text{slope} = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^{n} (x_i - \bar{x})^2}
$$
$$
\text{intercept} = \bar{y} - \text{slope} \times \bar{x}
$$
where $x_i$ is the GDP per capita of the $i$-th record, $y_i$ is the difference in labor force participation rate of the $i$-th record, $\bar{x}$ is the average GDP per capita, and $\bar{y}$ is the average difference in labor force participation rate.

```python
def compute_linear_regression_1(data):
    """
    Fit a linear regression line to the data
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    OUT: float, intercept of the regression line
         float, slope of the regression line
    """
    pass
```

In [ ]:
                              #as discussed after lecture, added optional param for ease of computing for other factors other than gdp_per_capita
def compute_linear_regression_1(data,param="gdp_per_capita"):
    """
    Fit a linear regression line to the data
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    OUT: float, intercept of the regression line
         float, slope of the regression line
    """
  
    #make all our data into np arrays
    x_obs = np.array(data[param])
    y_obs = np.array(data["difference_labor_force_participation_rate"])
    x_bar = np.mean(x_obs)
    y_bar = np.mean(y_obs)
    
    #formula
    num = np.sum(((x_obs -x_bar) * (y_obs - y_bar)))
    denom = np.sum(((x_obs -x_bar) **2))
    slope = num / denom
    intercept = y_bar - (slope * x_bar)

    return intercept,slope


In [ ]:
compute_linear_regression_1(cleaned_df)

##### Task 1.6

Write a Python function `plot_linear_regression_1()` that plots the data along with the linear regression line.

- The x-axis should be the GDP per capita.
- The y-axis should be the difference of the labor force participation rate.
- Use scatter points to plot the data and a line to represent the linear regression.

```python
def plot_linear_regression_1(data, slope, intercept):
    """
    Plot the data along with the linear regression line
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        slope, float, slope of the regression line
        intercept, float, intercept of the regression line
    OUT: None
    """
    pass
```

In [ ]:
def plot_linear_regression_1(data, slope, intercept):
    """
    Plot the data along with the linear regression line
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        slope, float, slope of the regression line
        intercept, float, intercept of the regression line
    OUT: None
    """
    #plot the scatter plot first
    x = data["gdp_per_capita"]
    y_scatter = data["difference_labor_force_participation_rate"]
    
    
    #plot the lin line
    y_lin = (np.array(x) * slope) + intercept
    
    scale = [4] * len(y_scatter)
    plt.figure(figsize=(14,8))
    plt.scatter(x,y_scatter,s=scale)
    plt.xlabel("gdp per capita")
    plt.ylabel("diff in male and female labor")
    plt.plot(x,y_lin,color="red")


     
   

In [ ]:
intercept,slope = compute_linear_regression_1(cleaned_df)
plot_linear_regression_1(cleaned_df, slope, intercept)


##### Task 1.7

Write a Python function `get_determinant_1()` that returns the coefficient of determination of the linear regression.

The coefficient of determination is calculated as follows:

$$
r^2 = \frac{\text{SSR}}{\text{SST}}
$$

where:
- $\text{SSR}$ (Sum of the Squared Residuals): The sum of the squared differences between the actual values and the predicted values.
- $\text{SST}$ (Total Sum of Squares): The sum of the squared differences between the actual values and their mean.

This metric measures how well the regression line fits the data, with values closer to 1 indicating a better fit.

```python
def get_determinant_1(data, slope, intercept):
    """
    Return the coefficient of determination of the linear regression
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        slope, float, slope of the regression line
        intercept, float, intercept of the regression line
    OUT: float, coefficient of determination
    """
    pass
```

In [ ]:
                                    #as discussed after lecture, added optional param for ease of computing for other factors other than gdp_per_capita
def get_determinant_1(data, slope, intercept,param="gdp_per_capita"):
    """
    Return the coefficient of determination of the linear regression
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        slope, float, slope of the regression line
        intercept, float, intercept of the regression line
    OUT: float, coefficient of determination
    """

    x = data[param]

    #observed 
    y_obs = np.array(data["difference_labor_force_participation_rate"])

    #predicted values
    y_pred = (np.array(x) * slope) + intercept

    #ssr
    ssr = np.sum((y_pred - np.mean(y_obs)) **2)

    #sst
    sst  = np.sum((y_obs -np.mean(y_obs))**2)


    r2 =(ssr/sst)    

    return r2

In [ ]:
intercept,slope = compute_linear_regression_1(cleaned_df)
get_determinant_1(cleaned_df, slope, intercept)

#### Level 2

In this level, we will examine whether, and to what extent, a country’s GDP per capita and the year are related to the difference in labor force participation rates between males and females.

##### Task 2.1

The same as Task 1.1.

```python
def get_data_2():
    """
    Retrieve the GDP per capita, labor force participation rate of each country/territory
    IN: None
    OUT: DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    """
    return get_data_1()
```

In [ ]:
def get_data_2():
    """
    Retrieve the GDP per capita, labor force participation rate of each country/territory
    IN: None
    OUT: DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    """
    return get_data_1()

##### Task 2.2

The same as Task 1.2.

```python
def save_data_2(data, mydb, cursor, db_name):
    """
    Save the data in a MySQL database
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: None
    """
    return save_data_1(data, mydb, cursor, db_name)
```

In [ ]:
def save_data_2(data, mydb, cursor, db_name):
    """
    Save the data in a MySQL database
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: None
    """
    return save_data_1(data, mydb, cursor, db_name)

##### Task 2.3

The same as Task 1.3.

```python
def load_data_2(mydb, cursor, db_name):
    """
    Query the data from the MySQL database
    IN: mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    """
    return load_data_1(mydb, cursor, db_name)
```

In [ ]:
def load_data_2(mydb, cursor, db_name):
    """
    Query the data from the MySQL database
    IN: mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    """
    return load_data_1(mydb, cursor, db_name)

##### Task 2.4

Write a Python function `plot_data_2()` that visualizes the relationship between GDP per capita, the year, and the difference in labor force participation rates using a 3D scatter plot.

- The x-axis should be the GDP per capita.
- The y-axis should be the year
- The z-axis should be the difference of the labor force participation rate.

Note: 
- You can use the following code to create a 3D scatter plot:
    ```python
    import matplotlib.pyplot as plt

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    ax.scatter(X, Y, Z, c='r', marker='o')

    ax.set_xlabel('GDP per capita')
    ax.set_ylabel('Year')
    ax.set_zlabel('Difference of labor force participation rate')

    plt.show()
    ```

```python
def plot_data_2(data):
    """
    Visualize the relationship between GDP per capita, the year, and the difference in labor force participation rates
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    OUT: None
    """
    pass
```

In [ ]:
def plot_data_2(data):
    """
    Visualize the relationship between GDP per capita, the year, and the difference in labor force participation rates
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    OUT: None
    """

    #grab the right series
    x = data["gdp_per_capita"]
    y = data["year"]
    z = data["difference_labor_force_participation_rate"]
    
    fig = plt.figure(figsize=(10,8))
    
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x,y,z)
    #set appropriate x,y, and z labels
    ax.set_xlabel('gdp per capita')
    ax.set_ylabel('year')
    ax.set_zlabel('difference of labor force participation rate')
    pass

In [ ]:
plot_data_2(cleaned_df)

##### Task 2.5

Write a Python function `compute_linear_regression_2()` similar to Task 1.5 except that it should fit a linear regression plane from both the GDP per capita and the year to the difference of the labor force participation rate. Your function should return slopes for GDP per capita and year and the intercept of the regression plane.

The linear regression plane can be found by minimizing the sum of the squared differences between the observed and predicted values. The slopes and intercept can be calculated as follows:

$$
\text{slope}_1 = \frac{\sum_{i=1}^{n} (x_{1,i} - \bar{x}_{1})(y_i - \bar{y})}{\sum_{i=1}^{n} (x_{1,i} - \bar{x}_{1})^2}
$$
$$
\text{slope}_2 = \frac{\sum_{i=1}^{n} (x_{2,i} - \bar{x}_{2})(y_i - \bar{y})}{\sum_{i=1}^{n} (x_{2,i} - \bar{x}_{2})^2}
$$
$$
\text{intercept} = \bar{y} - \text{slope}_1 \times \bar{x}_1 - \text{slope}_2 \times \bar{x}_2
$$
where $x_{1,i}$ is the GDP per capita of the $i$-th record, $x_{2,i}$ is the year of the $i$-th record, $y_i$ is the difference in labor force participation rate of the $i$-th record, $\bar{x}_1$ is the mean GDP per capita, $\bar{x}_2$ is the mean year, and $\bar{y}$ is the mean difference in labor force participation rate.

You can rewrite the above equations in matrix form as follows:
$$
\begin{bmatrix}
\text{intercept} \\
\text{slope}_1 \\
\text{slope}_2 \\
\end{bmatrix}
=
(X^T X)^{-1} X^T y
$$
where $X$ is a matrix of shape $(n, 3)$ with the first column being all ones, the second column being the GDP per capita, and the third column being the year, and $y$ is a vector of shape $(n,)$ with the difference in labor force participation rate.

```python
def compute_linear_regression_2(data):
    """
    Fit a linear regression plane to the data
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    OUT: ndarray of shape (3,), (intercept, slope_gdp, slope_year)
    """
    pass
```

In [ ]:
                                #as discussed after lecture, added optional params for ease of computing for other factors other than gdp_per_capita & year
def compute_linear_regression_2(data,params=("gdp_per_capita","year")):
    """
    Fit a linear regression plane to the data
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
    OUT: ndarray of shape (3,), (intercept, slope_gdp, slope_year)
    """
    n = len(data)

    #unpack added params
    p1,p2 = params 
    #change to axis =1 no point of using transpose to align intercept col
    X = np.stack([np.ones(n),data[p1],data[p2]],axis=1)
    y= data["difference_labor_force_participation_rate"]
    
    intercept_slopex_slopey= np.matmul(np.linalg.inv(np.matmul(X.T,X)), np.matmul(X.T,y)) 
    return intercept_slopex_slopey

In [ ]:
compute_linear_regression_2(cleaned_df)

##### Task 2.6

Write a Python function `plot_linear_regression_2()` that visualizes the data as a 3D scatter plot along with the fitted linear regression plane.

Note: 
- You can use the following code to create a plane in a 3D plot:
    ```python
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    Z = slope_gdp * X + slope_year * Y + intercept

    ax.plot_surface(X, Y, Z, alpha=0.5)

    plt.show()
    ```

```python
def plot_linear_regression_2(data, regression_params):
    """
    Visualize the data as a 3D scatter plot along with the fitted linear regression plane
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        regression_params, ndarray of shape (3,), (intercept, slope_gdp, slope_year)
    OUT: None
    """
    pass
```

In [ ]:
def plot_linear_regression_2(data, regression_params):
    """
    Visualize the data as a 3D scatter plot along with the fitted linear regression plane
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        regression_params, ndarray of shape (3,), (intercept, slope_gdp, slope_year)
    OUT: None
    """
    
    #destructure reg params
    intercept,slope_gdp,slope_year = regression_params
    #grab series
    x = data["gdp_per_capita"]
    y = data["year"]
    z = data["difference_labor_force_participation_rate"]
    
    #make meshgrid out of x y so we can plot 3d
    x_grid,y_grid = np.meshgrid(np.linspace(x.min(),x.max()),np.linspace(y.min(),y.max(),100))

    z_line = slope_gdp * x_grid + slope_year * y_grid +intercept
    

    #set up plot
    fig = plt.figure(figsize=(10,12))
    ax = fig.add_subplot(111, projection='3d')
    scale = [10] * len(y)
    ax.scatter(x,y,z,s=scale)
    ax.plot_surface(x_grid,y_grid,z_line,alpha=0.5,color="red")
    ax.set_xlabel('gdp per capita')
    ax.set_ylabel('year')
    ax.set_zlabel('diff of labor force participation rate')

    plt.show()
  

In [ ]:
regression_params = compute_linear_regression_2(cleaned_df)
plot_linear_regression_2(cleaned_df,regression_params)

##### Task 2.7

Write a Python function `get_determinant_2()` that calculates and returns the coefficient of determination for the linear regression plane fitted to predict the difference in labor force participation rates using GDP per capita and year as predictors.

```python
def get_determinant_2(data, regression_params):
    """
    Return the coefficient of determination of the linear regression plane
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        regression_params, ndarray of shape (3,), (intercept, slope_gdp, slope_year)
    OUT: float, coefficient of determination
    """
    pass
```

In [ ]:
                            #as discussed after lecture, added optional params for ease of computing for other factors other than gdp_per_capita & year
def get_determinant_2(data, regression_params,params=("gdp_per_capita","year")):
    """
    Return the coefficient of determination of the linear regression plane
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        regression_params, ndarray of shape (3,), (intercept, slope_gdp, slope_year)
    OUT: float, coefficient of determination
    """
    #destructure reg params & col params
    p1,p2 = params
   
    #grab series
    x = data[p1]
    y = data[p2]
    z_obs = data["difference_labor_force_participation_rate"]
    X= np.stack([np.ones(len(data)),x,y],axis=1)
    #prediction values
    z_pred = X @ regression_params

    #sum(( observed - predicted)^2)
    ssr = np.sum((z_pred - np.mean(z_obs)) **2)

    #sum((observed - mean(observed)**2))
    sst = np.sum((z_obs - np.mean(z_obs)) **2)




    r2 = (ssr/sst)
    return r2


In [ ]:
regression_params = compute_linear_regression_2(cleaned_df)
get_determinant_2(cleaned_df, regression_params)

##### Task 2.8

To find which factor is more important, write a Python function `compare_determinants_2()` to determine which factor—GDP per capita or year—is more influential in explaining the difference in labor force participation rates.

Steps:
- Calculate the "full model" determinant:
    Use `get_determinant_2()` to get the coefficient of determination for the linear regression plane with both GDP per capita and year as predictors.
- Calculate the "GDP model" determinant:
    Use `get_determinant_1()` to get the coefficient of determination for the linear regression with only GDP per capita as the predictor.
- Calculate the "year model" determinant:
    Use `get_determinant_1()` to get the coefficient of determination for the linear regression with only year as the predictor.
- Compare the differences:
    - Compute the difference between the "full model" determinant and the "GDP model" determinant.
    - Compute the difference between the "full model" determinant and the "year model" determinant.
- Determine the more important factor:
    - Return the factor (GDP per capita or year) associated with the larger difference.

```python
def compare_determinants_2(data, regression_params):
    """
    Determine which factor—GDP per capita or year—is more influential in explaining the difference in labor force participation rates
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        regression_params, ndarray of shape (3,), (intercept, slope_gdp, slope_year)
    OUT: str, the more important factor
    """
    pass
```

In [ ]:
def handle_single_determinant(data,param):
    intercept,slope = compute_linear_regression_1(data,param=param)
    return get_determinant_1(data,slope,intercept,param=param) 

def compare_determinants_2(data, regression_params):
    """
    Determine which factor—GDP per capita or year—is more influential in explaining the difference in labor force participation rates
    IN: data, DataFrame, data of GDP per capita, labor force participation rate of each country/territory
        regression_params, ndarray of shape (3,), (intercept, slope_gdp, slope_year)
    OUT: str, the more important factor
    """
    full_model = get_determinant_2(data,regression_params)

    #pass correct param name to the handle determinant function

    #gdp model
    gdp_model = full_model - handle_single_determinant(data,"gdp_per_capita")

    #year_model
    year_model = full_model - handle_single_determinant(data,"year")
   

    return "gdp_per_capita" if gdp_model < year_model else "year"

In [ ]:
regression_params = compute_linear_regression_2(cleaned_df)
compare_determinants_2(cleaned_df, regression_params)

#### Level 3

In this level, we will explore additional factors that may influence the difference in labor force participation rates between males and females.

##### Task 3.1

Write a Python function `get_data_3()` similar to Task 1.1 except that it should retrieve more data from the World Bank API:

- GDP per capita (current US$) (NY.GDP.PCAP.CD)
- Labor force participation rate, male (% of male population ages 15+) (modeled ILO estimate) (SL.TLF.CACT.MA.ZS)
- Labor force participation rate, female (% of female population ages 15+) (modeled ILO estimate) (SL.TLF.CACT.FE.ZS)
- **Labor force, female (% of total labor force) (SL.TLF.TOTL.FE.ZS)**
- **Literacy rate, adult male (% of males ages 15 and above) (SE.ADT.LITR.MA.ZS)**
- **Literacy rate, adult female (% of females ages 15 and above) (SE.ADT.LITR.FE.ZS)**
- **Life expectancy at birth, male (years) (SP.DYN.LE00.FE.IN)**
- **Life expectancy at birth, female (years) (SP.DYN.LE00.FE.IN)**

Note: You can decide to add more factors if you think they are relevant. A full list of available indicators can be found at https://data.worldbank.org/indicator?tab=all

```python
def get_data_3(x_indicator_codes):
    """
    Retrieve the data of each country/territory
    IN: x_indicator_codes, list of str, indicator codes as predictors for the linear regression
    OUT: DataFrame, data of each country/territory
    """
    pass
```

In [ ]:
def get_data_3(x_indicator_codes):
    """
    Retrieve the data of each country/territory
    IN: x_indicator_codes, list of str, indicator codes as predictors for the linear regression
    OUT: DataFrame, data of each country/territory
    """
    country_name_id = get_all_countries()
    data = {}
    indicators_dict = {
                    #"NY.GDP.PCAP.CD":"gdp",
                    #"SL.TLF.CACT.MA.ZS":"ML",
                    #"SL.TLF.CACT.FE.ZS":"FL",
                    "SL.TLF.TOTL.FE.ZS": "LFF",
                    "SE.ADT.LITR.MA.ZS": "LRM",
                    "SE.ADT.LITR.FE.ZS": "LRF",
                    "SP.DYN.LE00.MA.IN": "LEM",
                    "SP.DYN.LE00.FE.IN": "LEF"
                 }
    for j, (name,id)  in enumerate(country_name_id): #use the id to get gdp per capital 
        data[id] = [{
            "name":name,
            "id":id,
            "date":None,
            #"gdp":None, # not repeating same data as inserting into same database
            #"ML":None,
            #"FL":None,
            "LFF":None,
            "LRM":None,
            "LRF":None,
            "LEM":None,
            "LEF":None   
        }]
        for indicator in x_indicator_codes:
            country_data = req_helper(f"https://api.worldbank.org/v2/country/{id}/indicator/{indicator}?format=json&per_page=5000")
            if country_data != None:
                #make sure the data is there to grab, then insert in the approporiate dictionary
                if len(data[id]) != len(country_data):
                        data[id].extend([{"name":name,"id":id,"date":None,"gdp":None,"ML":None,"FL":None,"LFF":None,"LRM":None,"LRF":None,"LEM":None,"LEF":None} for _ in range(len(country_data) - len(data[id]))])
                for i,obj in enumerate(country_data):
                    if obj.get("date") and data[id][i]["date"] == None:
                        data[id][i]["date"] = obj["date"]
                    if obj.get("value"):
                        #use the dict to get the column value for the df
                        data[id][i][indicators_dict[indicator]] = obj["value"]
        #if j%10 == 0 : (for progress)
        #    print(j)

    #absolutely no data was found not surre if we should extend or delete the data entirely...(extending 4 now)
    cp = data.copy()
    length =  0
    for id,list_of_dicts in list(cp.items()):
        if len(list_of_dicts) > 1 and length ==0:
            length = len(list_of_dicts)
        if len(list_of_dicts) == 1:
            data[id] *= length
            
    list_of_data = [d for l in list(data.values()) for d in l]
 
    return pd.DataFrame(list_of_data)
    

In [ ]:
# using the same databse, so there is no point of repeating the same data
x_indicator_codes = [
    #"NY.GDP.PCAP.CD",
    #"SL.TLF.CACT.MA.ZS",
    #"SL.TLF.CACT.FE.ZS", 
    "SL.TLF.TOTL.FE.ZS",
    "SE.ADT.LITR.MA.ZS",
    "SE.ADT.LITR.FE.ZS",
    "SP.DYN.LE00.MA.IN",
    "SP.DYN.LE00.FE.IN"
    ]
#takes ~5.5 min
large_data =get_data_3(x_indicator_codes)

##### Task 3.2

Write a Python function `save_data_3()` similar to Task 1.2 except that it should save the additional data in the MySQL database.

```python
def save_data_3(data, mydb, cursor, db_name):
    """
    Save the data in a MySQL database
    IN: data, DataFrame, data of each country/territory
        mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: None
    """
    pass
```

In [ ]:
def create_remaining_tables():

    #value is percentage for labor force female
    create_labor_force_total_female = '''
    create table labor_force_total_female (
    country_code char(3) not null,
    year int,
    value double,
    foreign key (country_code) references country(code)  
    );
    '''

    #value is percentage
    create_literacy_rate = '''
    create table literacy_rate (
    country_code char(3) not null,
    year int,
    male_value double,
    female_value double,
    foreign key (country_code) references country(code) 
    );
    '''

    #value is years
    create_life_expectancy_rate = '''
    create table life_expectancy (
    country_code char(3) not null,
    year int,
    male_value double,
    female_value double,
    foreign key (country_code) references country(code)
    );
    '''
    return [create_labor_force_total_female,create_literacy_rate,create_life_expectancy_rate]


def save_data_3(data, mydb, cursor, db_name):
    """
    Save the data in a MySQL database
    IN: data, DataFrame, data of each country/territory
        mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: None
    """

     #********HANDLES CREATION OF TABLES********#
    create_statements =  create_remaining_tables()
    #give it a few seconds so the country table can  be made so the other tables can reference it, 5 secs might b overkill  but doenst matter.
    for q in create_statements:
        cursor.execute(q) 
        time.sleep(2)
    #********HANDLES CREATION OF TABLES********#


    #********HANDLES INSERTION INTO TOTAL LABOR FEMALE********#
    labor_total_female = data[["id","date","LFF"]].values
    for id,year,value in labor_total_female:
        q= f'''
        insert into labor_force_total_female (country_code,year,value) values ("{id}",{'null' if pd.isnull(year) else year}, {'null' if pd.isnull(value) else value});
        '''
        cursor.execute(q) 
    #********HANDLES INSERTION INTO TOTAL LABOR FEMALE********#


    #********HANDLES INSERTION INTO LITERACY_RATE********#
    filtered_df = data[["id","date","LRM","LRF"]]
    for _,row in filtered_df.iterrows():
        q= f'''
            insert into literacy_rate (country_code, year, male_value,female_value) values ('{row["id"]}',{'null' if pd.isnull(row["date"]) else row["date"]},{'null' if pd.isnull(row["LRM"]) else row["LRM"]},{'null' if pd.isnull(row["LRF"]) else row["LRF"]});
            '''
        cursor.execute(q)
    #********HANDLES INSERTION INTO LITERACY_RATE********#

    #********HANDLES INSERTION INTO LIFE_EXPECTANCY********#
    filtered_df = data[["id","date","LEM","LEF"]]
    for _,row in filtered_df.iterrows():
            q = f"""
            insert into life_expectancy (country_code, year, male_value,female_value) values ('{row["id"]}',{'null' if pd.isnull(row["date"]) else row["date"]},{'null' if pd.isnull(row["LEM"]) else row["LEM"]}, {'null' if pd.isnull(row["LEF"]) else row["LEF"]});
            """
            cursor.execute(q)
    #********HANDLES INSERTION INTO LIFE_EXPECTANCY********#

    mydb.commit()

In [ ]:
save_data_3(large_data, mydb, cursor, db_name)

##### Task 3.3

Write a Python function `load_data_3()` similar to Task 1.3 except that it should query the additional data from the MySQL database.

```python
def load_data_3(mydb, cursor, db_name):
    """
    Query the data from the MySQL database
    IN: mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: DataFrame, data of each country/territory
    """
    pass
```

In [ ]:
def load_data_3(mydb, cursor, db_name):
    """
    Query the data from the MySQL database
    IN: mydb, MySQL connection
        cursor, MySQL cursor
        db_name, str, name of the database
    OUT: DataFrame, data of each country/territory
    """
    pass 
    #large conditional query since were only grabbing rows where there is no null values for any cols
    #essentially join all tables, check if anything is null, then calculate the difference between the gender for partipation rate
    q = f"""
    select C.name, C.code, G.year, G.value as "gdp", (L.male_value - L.female_value) as 'difference_labor_force_participation_rate', 
    LFTF.value as "labor_force_total_female", LR.male_value,LR.female_value,LE.male_value, LE.female_value
    from country C 
    inner join gdp_per_capita G on C.code = G.country_code
    inner join labor_force_participation_rate L on C.code = L.country_code

    inner join labor_force_total_female LFTF on C.code = LFTF.country_code 
    inner join literacy_rate LR on C.code = LR.country_code 
    inner join life_expectancy LE on C.code = LE.country_code
    
    where G.year is not null and 
    G.value is not null and 
    L.male_value is not null and 
    L.female_value is not null and
    LE.male_value is not null and
    LE.female_value is not null and 
    LR.male_value is not null and 
    LR.female_value is not null and
    LFTF.value is not null and 
    G.year = L.year and 
    G.year = LFTF.year and
    G.year = LR.year and
    G.year = LE.year
    """
    cursor.execute(q)
    res = cursor.fetchall()
    data = []
    #parse the data in res, including all necessary columns for new cleaned dataframe
    for name,id,year,gdp,diff_lf_m_f,total_female_lf,lrm,lrf,lem,lef in res:
        d ={
            "country_name":name,
            "country_code":id,
            "year":year,
            "gdp_per_capita":gdp,
            "difference_labor_force_participation_rate":diff_lf_m_f,
            "labor_force_total_female":total_female_lf,
            "literacy_rate_m": lrm,
            "literacy_rate_f":lrf,
            "life_expectancy_m": lem,
            "life_expectancy_f":lef,
        }
        data.append(d)

    cleaned_df = pd.DataFrame(data)
    return cleaned_df

In [ ]:
cleaned_df_3 = load_data_3(mydb, cursor, db_name)

##### Task 3.4

Write a Python function `plot_data_3()` that visualizes the pairwise relationships between each factor and the difference in labor force participation rate.

Your plots should be 2D scatter plots for each factor against the difference in labor force participation rate.

```python
def plot_data_3(data):
    """
    Visualize the pairwise relationships between each factor and the difference in labor force participation rate
    IN: data, DataFrame, data of each country/territory
    OUT: None
    """
    pass
```

In [ ]:
def plot_data_3(data):
    """
    Visualize the pairwise relationships between each factor and the difference in labor force participation rate
    IN: data, DataFrame, data of each country/territory
    OUT: None
    """
    import matplotlib.pyplot as plt
    import math
    
    # get param cols
    params = [col for col in data.columns 
                     if col not in ['country_name', 'country_code', 
                                  'difference_labor_force_participation_rate']]
    
    #create enough plots for params
    n_plots = len(params)
    n_rows = math.ceil(n_plots / 2)  
    
    # gotta put figize pretty big or subplots get smuged
    fig, axes = plt.subplots(n_rows, 2, figsize=(15, 5*n_rows))
    fig.suptitle('Relationships between Factors and Difference in Labor Force Participation Rate', 
                fontsize=12, y=1)
    
    # flatten arr so iterating is ez
    axes = axes.flatten()
    
    # create scatter plots with specific series as x 
    for i, predictor in enumerate(params):
        ax = axes[i]
        
        # create scatter plot
        ax.scatter(data[predictor], 
                  data['difference_labor_force_participation_rate'],
                  alpha=0.5,
                  )
        
        # add percents 
        ax.set_xlabel(predictor  + " (%)" if predictor in ["labor_force_total_female", "literacy_rate_m","literacy_rate_f"] else predictor)
        ax.set_ylabel('Difference in Labor Force Participation Rate' if i % 2 == 0 else '')
        ax.set_title(f'{predictor} vs Difference in Labor Force Participation Rate')
        
    #get rid of plots not used
    for i in range(n_plots, len(axes)):
        fig.delaxes(axes[i])
    
    plt.tight_layout() # looks better
    plt.show()

In [ ]:
plot_data_3(cleaned_df_3)

##### Task 3.5

Write a Python function `compute_linear_regression_3()` that fits a high-dimensional hyperplane to model the relationship between all factors and the difference in labor force participation rate. Your function should return the slopes and intercept of the high-dimensional hyperplane.

The high-dimensional hyperplane can be found by minimizing the sum of the squared differences between the observed and predicted values. The slopes and intercept can be calculated as follows:

$$
\begin{bmatrix}
\text{intercept} \\
\text{slope}_1 \\
\text{slope}_2 \\
\vdots \\
\text{slope}_n \\
\end{bmatrix}
=
(X^T X)^{-1} X^T y

$$
where $X$ is a matrix of shape $(n, m+1)$ with the first column being all ones, and the remaining columns being the values of the factors, and $y$ is a vector of shape $(n,)$ with the difference in labor force participation rate.

```python
def compute_linear_regression_3(data):
    """
    Fit a high-dimensional hyperplane to model the relationship between all factors and the difference in labor force participation rate
    IN: data, DataFrame, data of each country/territory
    OUT: ndarray of shape (m+1,), (intercept, slope_1, slope_2, ..., slope_n)
    """
    pass
```

In [ ]:
def compute_linear_regression_3(data):
    """
    Fit a high-dimensional hyperplane to model the relationship between all factors and the difference in labor force participation rate
    IN: data, DataFrame, data of each country/territory
    OUT: ndarray of shape (m+1,), (intercept, slope_1, slope_2, ..., slope_n)
    """
    pass
    n = len(data)
    #get each col
    params = [data[col] for col in data.columns 
                if col not in ['country_name', 'country_code',  'difference_labor_force_participation_rate']
             ]
    #put the col of 1s in the beginning 
    params.insert(0,np.ones(n))
    X = np.stack(params,axis=1)

    y= data["difference_labor_force_participation_rate"]
    
    intercept_slopen= np.matmul(np.linalg.inv(np.matmul(X.T,X)), np.matmul(X.T,y)) 
    return intercept_slopen

In [ ]:
compute_linear_regression_3(cleaned_df_3)

##### Task 3.6

Similar to Task 1.7, write a Python function `get_determinant_3()` that returns the coefficient of determination of the linear regression hyperplane.

```python
def get_determinant_3(data, regression_params):
    """
    Return the coefficient of determination of the linear regression hyperplane
    IN: data, DataFrame, data of each country/territory
        regression_params, ndarray of shape (m+1,), (intercept, slope_1, slope_2, ..., slope_n)
    OUT: float, coefficient of determination
    """
    pass
```

In [ ]:
def get_determinant_3(data, regression_params):
    """
    Return the coefficient of determination of the linear regression hyperplane
    IN: data, DataFrame, data of each country/territory
        regression_params, ndarray of shape (m+1,), (intercept, slope_1, slope_2, ..., slope_n)
    OUT: float, coefficient of determination
    """
    #real z values
    z_obs = data["difference_labor_force_participation_rate"]
    #get correct cols
    params = [data[col] for col in data.columns 
            if col not in ['country_name', 'country_code', 'difference_labor_force_participation_rate']
         ]
    #col of 1 4 intercept
    params.insert(0, np.ones(len(data)))
    #stack along y axis
    X = np.stack(params,axis=1)
    #dot product, get the same result by doing regression_params[1:] * x1,x2,x3... 
    z_pred = X @ regression_params
    #calculate
    ssr = np.sum((z_pred - np.mean(z_obs)) **2)
    sst = np.sum((z_obs - np.mean(z_obs)) **2)
    r2 = (ssr/sst)
    return r2
    

In [ ]:
regression_params = compute_linear_regression_3(cleaned_df_3)
get_determinant_3(cleaned_df_3, regression_params)

##### Task 3.7

Similar to Task 2.8, write a Python function `compare_determinants_3()` that compares the importance of each factor in explaining the difference in labor force participation rate using the coefficient of determination. Your function should return the factors in non-increasing order of importance.

```python
def compare_determinants_3(data, regression_params):
    """
    Determine the two most important factors in explaining the difference in labor force participation rate
    IN: data, DataFrame, data of each country/territory
        regression_params, ndarray of shape (m+1,), (intercept, slope_1, slope_2, ..., slope_n)
    OUT: list of int, the indices of the orderings of the factors in non-increasing order of importance
    """
```

In [ ]:
def compare_determinants_3(data, regression_params):
    """
    Determine the two most important factors in explaining the difference in labor force participation rate
    IN: data, DataFrame, data of each country/territory
        regression_params, ndarray of shape (m+1,), (intercept, slope_1, slope_2, ..., slope_n)
    OUT: list of int, the indices of the orderings of the factors in non-increasing order of importance
    """
   
    # get the params from data
    params = [(i,col) for i,col in enumerate(data.columns) if col not in ['country_name', 'country_code', 'difference_labor_force_participation_rate']]
        
    
    full_model = get_determinant_3(data,regression_params)
    
    model_diff = sorted([(full_model - handle_single_determinant(data,param),i) for i,param in params],reverse=False,key=lambda x:x[0] )
 
    return [i for _,i in model_diff]    




In [ ]:
regression_params= compute_linear_regression_3(cleaned_df_3)
compare_determinants_3(cleaned_df_3, regression_params)

##### Task 3.8

Take the most two important factors and write a Python function `plot_linear_regression_3()` that produces a 3D plot of the data and the linear regression plane using the two most important factors.

- The x-axis should be the most important factor.
- The y-axis should be the second most important factor
- The z-axis should be the difference of the labor force participation rate.

```python
def plot_linear_regression_3(data, regression_params, important_factors):
    """
    Visualize the data as a 3D scatter plot along with the fitted linear regression plane
    IN: data, DataFrame, data of each country/territory
        regression_params, ndarray of shape (m+1,), (intercept, slope_1, slope_2, ..., slope_n)
        important_factors, (int, int), the indices of the two most important factors
    OUT: None
    """
    pass
```

In [ ]:
def plot_linear_regression_3(data, regression_params, important_factors):
    """
    Visualize the data as a 3D scatter plot along with the fitted linear regression plane
    IN: data, DataFrame, data of each country/territory
        regression_params, ndarray of shape (m+1,), (intercept, slope_1, slope_2, ..., slope_n)
        important_factors, (int, int), the indices of the two most important factors
    OUT: None
    """ 

    #get 2 most important factor indicies
    index0,index1 = important_factors[:2]
    #get col names
    col0_name = data.columns[index0]
    col1_name = data.columns[index1]

    #grab series
    x = data[col0_name]
    y= data[col1_name]
    z = data["difference_labor_force_participation_rate"]

    #calcuate slope & intercept
    intercept,slope_x,slope_y = compute_linear_regression_2(data,(col0_name,col1_name))

    x_grid,y_grid = np.meshgrid(np.linspace(x.max(),x.min()),np.linspace(y.max(),y.min(),100))

    z_line =intercept + slope_x * x_grid + slope_y * y_grid 
    

    #set up plot
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(x,y,z,alpha=1)
    #plot plane
    ax.plot_surface(X=x_grid,Y=y_grid,Z=z_line,alpha=0.5,color="red")

    #labels
    ax.set_xlabel(data.columns[index0])
    ax.set_ylabel(data.columns[index1])
    
    #set lim so plane doesnt extend beyond possible values
    ax.set_xlim([x.min(), x.max()])
    ax.set_ylim([y.min(), y.max()])
    #ax.set_zlim([z.min(), z.max()])
    ax.view_init(azim=90)
    ax.set_zlabel('Difference of labor force participation rate')





In [ ]:
important_factors = compare_determinants_3(cleaned_df_3, regression_params)
plot_linear_regression_3(cleaned_df_3, regression_params, important_factors)

In [ ]:
#UI helper

def test_model(button_id,input):
    '''
    IN:
        button_id: int, index of the param model user clicked on
        inputs: inputs for each param
    OUT:
        (int) predicted value for that model
    '''

    #assertions
    assert type(button_id) is int
    assert type(input) is (list)

    #matching model to button id then predicting based off regression params y=mx+b for single, dot product for multiple params
    match button_id:
        case 0:
            input = input[0]
            intercept, slope = compute_linear_regression_1(cleaned_df)
            return  ((input*slope)+intercept) 
        case 1:
            regression_params = compute_linear_regression_2(cleaned_df)
            input.insert(0,1)
            input = np.array(input)
            return (input @ regression_params)
        case 2:
            regression_params = compute_linear_regression_3(cleaned_df_3)
            input.insert(0,1)
            input = np.array(input)
            return (input @ regression_params)
        case 3:
            input.insert(0,1)
            input = np.array(input)
            regression_params = compute_linear_regression_2(cleaned_df_3,("labor_force_total_female","life_expectancy_m"))
            return (input @ regression_params)
        case _:
            raise Error("sum went wrong.")
    


In [ ]:
#WARNING: ui is a bit buggy, make sure to terminate the tk tab rather stopping the process in jupyter, causes py kernel to crash...
# allows it to be launched in jupyter env
%gui tk

inputs = {"button":None}

def show_inputs(button_id):
    global inputs
    
    #reset the result everytime a button is clicked
    result_label.config(text=f"Result:")

    #when they click on a new button for a different model get rid of the other widgets packed into the main app
    for widget in input_frame.winfo_children():
        widget.destroy()

    #single gdp model
    if button_id == 0:
        inputs = {"button":button_id,"inputs":[]}
        tk.Label(input_frame, text="gdp").pack()
        entry = tk.Entry(input_frame)
        entry.pack()
        #gna store all the inputs inside of input["inputs"]
        inputs["inputs"].append(entry)
    #dual paramter gdp,year model
    elif button_id == 1:
        inputs = {"button":button_id,"inputs":[]}
        tk.Label(input_frame, text="gdp,year").pack()
        for _ in range(2):
            entry = tk.Entry(input_frame)
            entry.pack()
            inputs["inputs"].append(entry)
    #full seven param model
    elif button_id == 2:
        inputs = {"button":button_id,"inputs":[]}
        tk.Label(input_frame, text="gdp, year, labor force total female (rate), literacy rate male (rate), literacy rate female, (rate) life expectancy male (years), life expectancy female (years)").pack()
        for _ in range(7):
            entry = tk.Entry(input_frame)
            entry.pack()
            inputs["inputs"].append(entry)
    #most important factor model
    elif button_id == 3:
        inputs = {"button":button_id,"inputs":[]}
        tk.Label(input_frame, text="labor force total female (ex:44 for 44%), life expectancy male (years)").pack()
        for _ in range(2):
            entry = tk.Entry(input_frame)
            entry.pack()
            inputs["inputs"].append(entry)
           

def handle_submit():

    #find out which function were gna use based off the button id stored in inputs
    button_id = inputs["button"]
    try:
        # get values user input and compute prediction
        values = [float(entry.get()) for entry in list(inputs["inputs"])]
        result = test_model(button_id,values)
        result_label.config(text=f"Result: {result} is the difference in labor force participation rates")
    except:
        result = "please insert an int or float"
        result_label.config(text=f"Result: {result}")


# window
app = tk.Tk()
app.title("linear regression")

# frame for the buttons
button_frame = tk.Frame(app)
button_frame.pack(pady=10)

# make buttons based off index, will help in handling function logic
button_names = ["one param model (gdp)", "two param model (gdp,year)", "seven param model", "two param model (most important)"]
for i in range(0, 4):
    tk.Button(button_frame, text=f"{button_names[i]}", command=lambda i=i: show_inputs(i)).pack()

# frame for inputs
input_frame = tk.Frame(app)
input_frame.pack(pady=10)



#submit, dont know how to make it not show until all fields are filled
submit_button = tk.Button(app, text="submit", command=handle_submit)
submit_button.pack()


result_frame = tk.Frame(app)
result_frame.pack()
result_label = tk.Label(result_frame, text="choose a model and enter param values")
result_label.pack()


app.mainloop()


In [ ]:
#close all plots since were using interactive & close cursor/db connection
plt.close("all")
cursor.close()
mydb.close()

### References

> - Rencher, Alvin C.; Christensen, William F. (2012), "Chapter 10, Multivariate regression – Section 10.1, Introduction", Methods of Multivariate Analysis, Wiley Series in Probability and Statistics, vol. 709 (3rd ed.), John Wiley & Sons, p. 19, ISBN 9781118391679